In [1]:
import pymysql
import os
import pandas as pd
import datetime

In [2]:
folder_path = './v1.5'

In [3]:
db = pymysql.connect(host='localhost',
                     user='root',
                     password='Czy026110',
                     database='homework')
cursor = db.cursor()
sql = "INSERT INTO project_superset (issue_number, internal_issue_number, username, created_at, ended_at, is_pull_request, labels, project_name, version_number, content, positive_score, negative_score) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

In [4]:
for filename in os.listdir(folder_path):
    if filename == '.DS_Store':
        continue
    if filename.startswith('FORMAT'):
        continue
    file_path = os.path.join(folder_path,filename)
    project_name = ''
    version_number = ''
    issue_number = ''
    username = ''
    created_at = ''
    ended_at = ''
    is_pull_request = -1
    labels = ''
    if os.path.isfile(file_path):
        print
        with open(file_path, 'r') as file:
            lines = file.readlines()
            project_name = lines[1].replace('\r','').replace('\n','')
            version_number = lines[2].replace('\r','').replace('\n','')
            issue_number = lines[3].replace('\r','').replace('\n','')
            username = lines[4].replace('\r','').replace('\n','')
            created_at = lines[5].replace('\r','').replace('\n','')
            cdt = datetime.datetime.strptime(created_at, "%Y-%m-%dT%H:%M:%SZ")
            cdate = cdt.date()
            ctime = cdt.time()
            created_at = str(cdate) + ' ' + str(ctime)
            ended_at = lines[6].replace('\r','').replace('\n','')
            if ended_at != 'None':
                edt = datetime.datetime.strptime(ended_at, "%Y-%m-%dT%H:%M:%SZ")
                edate = edt.date()
                etime = edt.time()
                ended_at = str(edate) + ' ' + str(etime)
            if lines[7].replace('\r','').replace('\n','') == 'not_pull_request':
                is_pull_request = 0
            else:
                is_pull_request = 1
            labels = lines[8].replace('\r','').replace('\n','')
            file.close()

    result_filename = 'FORMAT' + filename.replace('.txt','') + '0_out.txt'
    result_file_path = os.path.join(folder_path,result_filename)

    if os.path.isfile(result_file_path):
        df = pd.read_csv(result_file_path,sep='\t',header=None,skiprows=[0])
        df.columns = ['Positive','Negative', 'Text']
        for index, row in df.iterrows():
            if index == 0:
                continue
            positive_score = row['Positive']
            negative_score = row['Negative']
            content = row['Text']
            internal_issue_number = index
            try:
                cursor.execute(sql, (issue_number, internal_issue_number, username, created_at, ended_at, is_pull_request, labels, project_name, version_number, content, positive_score, negative_score))
                db.commit()
            except:
               db.rollback()
db.close()